In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tqdm import tqdm

# Load the pretrained InceptionV3 model
def load_pretrained_model():
    base_model = InceptionV3(include_top=False, weights='imagenet', pooling='avg')
    model = Model(inputs=base_model.input, outputs=base_model.output)
    return model

pretrained_model = load_pretrained_model()

# Extract frames from a video
def extract_video_frames(video_path, sequence_length=16, image_width=299, image_height=299):
    frames_list = []
    video_reader = cv2.VideoCapture(video_path)
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count / sequence_length), 1)

    for frame_counter in range(sequence_length):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read()
        if not success:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        resized_frame = cv2.resize(frame_rgb, (image_height, image_width))
        frames_list.append(resized_frame)

    video_reader.release()
    return frames_list

# Extract features from frames using the pretrained model
def extract_frame_features(frame, pretrained_model):
    img = np.expand_dims(frame, axis=0)
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    feature_vector = pretrained_model.predict(img, verbose=0)
    return feature_vector

# Extract features from videos
def extract_features_from_videos(video_paths, pretrained_model):
    all_video_features = []
    for video_path in tqdm(video_paths):
        frames_list = extract_video_frames(video_path)
        frames_features = [extract_frame_features(frame, pretrained_model) for frame in frames_list]
        all_video_features.append(frames_features)
    return np.array(all_video_features)

# Define violence and non-violence directories
violence_dir = '/content/drive/MyDrive/archive (7)/Real Life Violence Dataset/Violence'
nonviolence_dir = '/content/drive/MyDrive/archive (7)/Real Life Violence Dataset/NonViolence'

# Create paths to individual videos
violence_path = [os.path.join(violence_dir, name) for name in os.listdir(violence_dir)]
nonviolence_path = [os.path.join(nonviolence_dir, name) for name in os.listdir(nonviolence_dir)]

# Extract features from videos
violence_features = extract_features_from_videos(violence_path[:500], pretrained_model)
non_violence_features = extract_features_from_videos(nonviolence_path[:500], pretrained_model)

# Save extracted features
np.save('/content/drive/MyDrive/archive (7)/real life violence situations/Violence_features.npy', violence_features)
np.save('/content/drive/MyDrive/archive (7)/real life violence situations/NonViolence_features.npy', non_violence_features)


 95%|█████████▌| 475/500 [59:53<02:37,  6.30s/it]

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Bidirectional, BatchNormalization, Dropout, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Load features and labels
violence_features = np.load('/content/drive/MyDrive/archive (7)/real life violence situations/Violence_features.npy')
non_violence_features = np.load('/content/drive/MyDrive/archive (7)/real life violence situations/NonViolence_features.npy')

# Creating labels
violence_labels = np.ones(len(violence_features))
non_violence_labels = np.zeros(len(non_violence_features))

# Combining features and labels
X = np.concatenate([violence_features, non_violence_features], axis=0)
y = np.concatenate([violence_labels, non_violence_labels], axis=0)

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Reshaping data for LSTM input
X_train_reshaped = X_train.reshape((X_train.shape[0], 16, 2048))
X_test_reshaped = X_test.reshape((X_test.shape[0], 16, 2048))

# Building the LSTM model
inputs = Input(shape=(16, 2048))
x = Bidirectional(LSTM(200, return_sequences=True))(inputs)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Bidirectional(LSTM(100))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(200, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=2, validation_data=(X_test_reshaped, y_test))

# Save the trained model
model.save('/content/drive/MyDrive/archive (7)/real life violence situations/violence_detection_model_sgd.h5')


NameError: name 'np' is not defined

In [ ]:
from google.colab import files
from tensorflow.keras.models import Model, load_model

# Load the trained model
model = load_model('/content/drive/MyDrive/archive (7)/real life violence situations/violence_detection_model_sgd.h5')

def extract_video_frames_for_prediction(video_path, sequence_length=16, image_width=299, image_height=299):
    frames_list = []
    video_reader = cv2.VideoCapture(video_path)
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count / sequence_length), 1)

    for frame_counter in range(sequence_length):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read()
        if not success:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        resized_frame = cv2.resize(frame_rgb, (image_height, image_width))
        frames_list.append(resized_frame)

    video_reader.release()
    return frames_list

def extract_frame_features_for_prediction(frame, pretrained_model):
    img = np.expand_dims(frame, axis=0)
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    feature_vector = pretrained_model.predict(img, verbose=0)
    return feature_vector

def predict_violence_in_video(video_path, pretrained_model, model):
    frames = extract_video_frames_for_prediction(video_path)
    frames_features = [extract_frame_features_for_prediction(frame, pretrained_model) for frame in frames]
    frames_features = np.array(frames_features).reshape((1, 16, 2048))

    prediction = model.predict(frames_features)
    return prediction[0][0]

# Upload a video file
uploaded = files.upload()

# Assuming the pretrained InceptionV3 model is available
pretrained_model = load_pretrained_model()

# Predict violence for each uploaded video
for video_filename in uploaded.keys():
    print(f'Processing video: {video_filename}')
    prediction = predict_violence_in_video(video_filename, pretrained_model, model)
    print(f'Prediction score for {video_filename}: {prediction}')
    if prediction > 0.5:
        print(f'Violence detected in {video_filename}')
    else:
        print(f'No violence detected in {video_filename}')
